In [1]:
from datetime import datetime
from tkinter import messagebox
import tkinter as tk
import pandas as pd
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore")

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24

año_mes = f"{año}{str(mes).zfill(2)}" # 202401

In [2]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir('bases/pagos/'+año_mes):
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(fecha_list[0] + '.' + fecha_list[1] + '.' + fecha_list[2], '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

'03.10.2024'

In [3]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    if not os.path.exists(carpeta_hoy+'/multi_agencias'):
        os.makedirs(carpeta_hoy+'/multi_agencias')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}\multi_agencias')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

La carpeta ya existe: reporte\2024.10.04
La carpeta ya existe: reporte\2024.10.04\multi_agencias


'reporte\\2024.10.04'

In [4]:
base_pagos_path = 'bases/pagos/' + año_mes + '/Base Pagos ' + fecha + '.xlsx'
asignacion_path = 'bases/asignacion/' + año_mes + '/base_asignacion_' + mes_año + '.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = file_path + '/MONOPRODUCTO ' + fecha + '.xlsx'
multiproducto = file_path + '/MULTIPRODUCTO ' + fecha + '.xlsx'
reactiva = file_path + '/REACTIVA ' + fecha + '.xlsx'
no_enviados = file_path + '/NO ENVIADOS ' + fecha + '.xlsx'
si_enviados = file_path + '/ENVIADOS ' + fecha + '.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)
enviados_path = os.path.abspath(si_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)
print(enviados_path)

bases/pagos/202410/Base Pagos 03.10.2024.xlsx
bases/asignacion/202410/base_asignacion_OCT24.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.10.04\MONOPRODUCTO 03.10.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.10.04\MULTIPRODUCTO 03.10.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.10.04\REACTIVA 03.10.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.10.04\NO ENVIADOS 03.10.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.10.04\ENVIADOS 03.10.2024.xlsx


In [5]:
#################### BASE DE PAGOS ####################

In [6]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(125, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-10-03,31009787,248,732,300.00,PEN,ACMED RAUL YALI FALCON,P
1,11,2024-10-03,20430460,248,229,2000.00,PEN,ANGEL RENE MAYTA .FLORES,P
2,11,2024-10-03,23949598,248,360,5222.80,PEN,NANCY LOURDES CANDIOTTI GONZALES,P
3,11,2024-10-03,25174819,248,175,3815.77,PEN,001101755002511031,P
4,11,2024-10-03,26564216,248,166,1000.00,PEN,ROSA HILDA MATTOS BEJAR,P


In [7]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base.rename(columns={'CODCEN': 'CC'}, inplace=True)

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(5)
df_base['CENTROPAGO'] = df_base['CENTROPAGO'].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(125, 10)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO         object
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FECHA_ENVIO,ID_RESPONSABLE
0,2024-10-03,31009787,00248,00732,300.00,PEN,ACMED RAUL YALI FALCON,P,04-Oct,MIV
1,2024-10-03,20430460,00248,00229,2000.00,PEN,ANGEL RENE MAYTA .FLORES,P,04-Oct,MIV
2,2024-10-03,23949598,00248,00360,5222.80,PEN,NANCY LOURDES CANDIOTTI GONZALES,P,04-Oct,MIV
3,2024-10-03,25174819,00248,00175,3815.77,PEN,001101755002511031,P,04-Oct,MIV
4,2024-10-03,26564216,00248,00166,1000.00,PEN,ROSA HILDA MATTOS BEJAR,P,04-Oct,MIV


In [8]:
#################### ASIGNACION ####################

In [9]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base de asignación?", icon='warning')
if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path)

In [10]:
print(df_asignacion.shape)
df_asignacion.head(5)

(243127, 8)


,CODIGO,CONT_18,NOMBRE,TIPO_CARTERA,TIPO_FONDO,CLAVE,AGENCIA,FLAG
0,22027135,1107969600048135,JUAN CARLOS CASTILLO DELGADO,SECURED,REACTIVA,KSTBC,SIN AGENCIA,2
1,27324374,1103339600373526,FERNANDO ORE CONSULTING EIRL,SECURED,REACTIVA,KSTBC,SIN AGENCIA,1
2,22027135,1107969600049018,JUAN CARLOS CASTILLO DELGADO,SECURED,REACTIVA,KSTBC,SIN AGENCIA,2
3,26916529,1101259600156222,GEOR SAC,SECURED,REACTIVA,KSTBC,SIN AGENCIA,3
4,23614464,1104269600064995,CLAUDIA JIMENEZ EIRL,SECURED,REACTIVA,KSTBC,SIN AGENCIA,1


In [11]:
#################### BACKUPS ####################

In [12]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

Base (125, 10)
Asignacion (243127, 8)


In [13]:
base_count = df_base_test.shape[0]
base_count

125

In [14]:
import numpy as np

def actualizar_tipo_cartera(grupo: pd.DataFrame) -> pd.DataFrame:
    grupo['TIPO_CARTERA'] = np.where(
        grupo['TIPO_CARTERA'].eq('UNSECURED').any(), 'UNSECURED',
        np.where(
            ~grupo['TIPO_CARTERA'].eq('NULL').any(), 'SECURED', 
            np.where(
                ~grupo['TIPO_CARTERA'].eq('SECURED').any(), 'NULL', 'SECURED/NULL'
            )
        )
    )
    return grupo

In [15]:
df_asignacion_test['CODIGO'] = df_asignacion_test['CODIGO'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test.rename(columns={'CONT_18': 'CONTRATO', 'NOMBRE': 'NOMBRE_CLIENTE', 'CLAVE': 'CARTERA', 'ID_FLAG': 'FLAG', 'AGENCIA_CORRECTA': 'AGENCIA'}, inplace=True)

cols_asignacion = ['CODIGO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA']
df_asignacion_test = df_asignacion_test[cols_asignacion]

df_base_test = df_base_test.merge(df_asignacion_test, left_on='CC', right_on='CODIGO', how='left')
df_base_test.drop(columns=['CODIGO'], inplace=True)
df_base_test.sort_values(by=['TIPO_CARTERA'], ascending=False, inplace=True)
df_base_test.sort_values(by=['CC'], ascending=True, inplace=True)

filtro = (df_base_test['FLAG'] != 1) & (df_base_test['FLAG'].notna())
df_base_filtro = df_base_test.loc[filtro].groupby('CC').apply(actualizar_tipo_cartera).reset_index(drop=True)
df_base_test = df_base_test.merge(df_base_filtro[['CC', 'TIPO_CARTERA']], on='CC', how='left', suffixes=('', '_y'))
df_base_test['TIPO_CARTERA_FINAL'] = df_base_test['TIPO_CARTERA_y'].fillna(df_base_test['TIPO_CARTERA'])

df_base_test.drop(columns=['TIPO_CARTERA', 'TIPO_CARTERA_y'], inplace=True)
df_base_test.rename(columns={'TIPO_CARTERA_FINAL': 'TIPO_CARTERA'}, inplace=True)
df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(125, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO         object
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                Int64
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-10-03,00099430,00248,00876,300.00,PEN,"OSCAR,RICARDO ARANA RONCEROS",P,1,001104085002240238,NULL,KSTBC,OSCAR RICARDO ARANA RONCEROS,04-Oct,MIV,UNSECURED,CLASA MORA
1,2024-10-03,02373823,00248,00263,999.95,PEN,L40849125 CC24654739 001102635000461221,P,<NA>,NaN,NULL,NaN,NaN,04-Oct,MIV,NULL,NaN
2,2024-10-03,02373823,00248,00263,0.05,PEN,L40849125 CC24654739 0011.02635000461221,P,<NA>,NaN,NULL,NaN,NaN,04-Oct,MIV,NULL,NaN
3,2024-10-03,13157693,00248,00192,2999.90,PEN,TC 00110192265001609987,P,1,001101925001609987,NULL,EXJ,ROLANDO OMAR ZEVALLOS LAZABARA,04-Oct,MIV,UNSECURED,SIN AGENCIA
4,2024-10-03,13361975,00248,00083,1100.00,PEN,"DANIA MARIETA SARDON SANCHEZ,,,",P,1,001102009600556058,NULL,BC,DANIA MARIETA SARDON SANCHEZ,04-Oct,MIV,SECURED,SIN AGENCIA


In [16]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

REGISTROS OK


In [17]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
no_flag_count = df_base_no_flag.shape[0]
print(no_flag_count)
df_base_no_flag.head(5)

4


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
1,2024-10-03,02373823,00248,00263,999.95,PEN,L40849125 CC24654739 001102635000461221,P,<NA>,NaN,NULL,NaN,NaN,04-Oct,MIV,NULL,NaN
2,2024-10-03,02373823,00248,00263,0.05,PEN,L40849125 CC24654739 0011.02635000461221,P,<NA>,NaN,NULL,NaN,NaN,04-Oct,MIV,NULL,NaN
28,2024-10-03,21665261,00249,00261,500.00,USD,CARLOS GUILLERMO CAMPOS DE LA PORTILLA,P,<NA>,NaN,NULL,NaN,NaN,04-Oct,MIV,NULL,NaN
156,2024-10-03,31357506,00248,00222,1400.00,PEN,PILCO FLORES MAGALI OFELIA,P,<NA>,NaN,NULL,NaN,NaN,04-Oct,MIV,NULL,NaN


In [18]:
#################### MONOPRODUCTO ####################

In [19]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
mono_count = df_mono.shape[0]
print(mono_count)
df_mono.head(5)

114


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-10-03,00099430,00248,00876,300.0,PEN,"OSCAR,RICARDO ARANA RONCEROS",P,1,001104085002240238,NULL,KSTBC,OSCAR RICARDO ARANA RONCEROS,04-Oct,MIV,UNSECURED,CLASA MORA
3,2024-10-03,13157693,00248,00192,2999.9,PEN,TC 00110192265001609987,P,1,001101925001609987,NULL,EXJ,ROLANDO OMAR ZEVALLOS LAZABARA,04-Oct,MIV,UNSECURED,SIN AGENCIA
4,2024-10-03,13361975,00248,00083,1100.0,PEN,"DANIA MARIETA SARDON SANCHEZ,,,",P,1,001102009600556058,NULL,BC,DANIA MARIETA SARDON SANCHEZ,04-Oct,MIV,SECURED,SIN AGENCIA
5,2024-10-03,14363181,00248,00317,575.0,PEN,ROSALIA GUZMAN MARTIN,P,1,001103665001112123,NULL,EXJ,ROSALIA MERCEDES GUZMAN MARTIN,04-Oct,MIV,UNSECURED,MORNESE MORA
6,2024-10-03,15475544,00248,00933,3000.0,PEN,ALLYSON CECILIA JARAMILLO ORTIZ,P,1,001101729600326051,NULL,BC,ALLYSON CECILIA JARAMILLO ORTIZ,04-Oct,MIV,UNSECURED,MORNESE MORA


In [20]:
#################### REACTIVA ####################

In [21]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'] != 'NULL')]
df_reactiva.sort_values(by=['FECHA', 'CC'], inplace=True)
df_reactiva.reset_index(drop=True, inplace=True)
df_reactiva.to_excel(reactiva, index=False)

reactiva_count = df_reactiva.shape[0]
print(reactiva_count)
df_reactiva.head(5)

4


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-10-03,20430460,00248,00229,2000.00,PEN,ANGEL RENE MAYTA .FLORES,P,1,001102299600426322,REACTIVA,BC,ANGEL RENE MAYTA FLORES,04-Oct,MIV,UNSECURED,CLASA MORA
1,2024-10-03,27692222,00248,00166,1000.00,PEN,"EMPAISLA EIRL,,,.",P,1,001101279600250750,REACTIVA,BC,EMPAISLA EIRL,04-Oct,MIV,UNSECURED,ASESCOM RJ
2,2024-10-03,27809125,00248,00141,5000.00,PEN,JIA CALLAO SAC,P,1,001101419600144698,REACTIVA,KSTBC,JIA CALLAO SAC,04-Oct,MIV,UNSECURED,CLASA MORA
3,2024-10-03,28686993,00248,00285,5067.39,PEN,PRODUCTOS DENTALES DEL NORTE,P,1,001107969600048364,REACTIVA,BC,PRODUCTOS DENTALES DEL NORTE EIRL,04-Oct,MIV,UNSECURED,SIN AGENCIA


In [22]:
#################### FIN REACTIVA ####################

In [23]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
no_mono_count

5

In [24]:
df_mono = df_mono[df_mono['TIPO_CARTERA'] == 'UNSECURED']
df_mono = df_mono[df_mono['TIPO_FONDO'] == 'NULL']

cols_mono = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA']
df_mono = df_mono[cols_mono]

df_mono.sort_values(by=['FECHA', 'CC'], inplace=True)
df_mono.reset_index(drop=True, inplace=True)
df_mono.to_excel(monoproducto, index=False)

mono_count_final = df_mono.shape[0]
print(mono_count_final)
df_mono.head(5)

105


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-10-03,00099430,00248,300.0,PEN,"OSCAR,RICARDO ARANA RONCEROS",P,1,001104085002240238,OSCAR RICARDO ARANA RONCEROS,04-Oct,MIV,UNSECURED,NULL,KSTBC,CLASA MORA
1,2024-10-03,13157693,00248,2999.9,PEN,TC 00110192265001609987,P,1,001101925001609987,ROLANDO OMAR ZEVALLOS LAZABARA,04-Oct,MIV,UNSECURED,NULL,EXJ,SIN AGENCIA
2,2024-10-03,14363181,00248,575.0,PEN,ROSALIA GUZMAN MARTIN,P,1,001103665001112123,ROSALIA MERCEDES GUZMAN MARTIN,04-Oct,MIV,UNSECURED,NULL,EXJ,MORNESE MORA
3,2024-10-03,15475544,00248,3000.0,PEN,ALLYSON CECILIA JARAMILLO ORTIZ,P,1,001101729600326051,ALLYSON CECILIA JARAMILLO ORTIZ,04-Oct,MIV,UNSECURED,NULL,BC,MORNESE MORA
4,2024-10-03,20124892,00248,290.0,PEN,NORITA SAAVEDRA DIAZ,P,1,001103015002014473,NORITA SAAVEDRA DIAZ,04-Oct,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ


In [25]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

MONOPRODUCTO OK


In [26]:
#################### MULTIPRODUCTO ####################

In [27]:
df_multi = df_base_test[df_base_test['FLAG'] > 1].copy()
multi_count = df_multi.shape[0]
print(multi_count)
df_multi.head(5)

7


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
9,2024-10-03,20275065,00248,00284,100.0,PEN,MARIA EUGENIA SALAZAR SALVATIERRA PAGO,P,2,001101279600302076,NULL,KSTBC,MARIA EUGENIA SALAZAR SALVATIERRA,04-Oct,MIV,UNSECURED,MORNESE MORA
40,2024-10-03,23589142,00248,00184,4500.0,PEN,OLIMPO DAVID CARPIO ALVARADO,P,2,001101999600443020,NULL,BC,OLIMPO DAVID CARPIO ALVARADO,04-Oct,MIV,UNSECURED,SIN AGENCIA
48,2024-10-03,24228176,00248,00083,1500.0,PEN,MARTHA FELICITAS OLIVERA VASQUEZ,P,2,001102009600379849,NULL,BC,MARTHA FELICITAS OLIVERA VASQUEZ,04-Oct,MIV,SECURED,SIN AGENCIA
66,2024-10-03,25975753,00248,00306,485.0,PEN,EDWIN ALEJANDRO PEREZ ZUMAETA 71531021*,P,2,001105045004232234,NULL,KSTBC,EDWIN ALEJANDRO PEREZ ZUMAETA,04-Oct,MIV,UNSECURED,CLASA MORA
70,2024-10-03,25975753,00248,00306,6067.0,PEN,EDWIN ALEJANDRO PEREZ ZUMAETA 71531021,P,2,001105049601205812,NULL,KSTBC,EDWIN ALEJANDRO PEREZ ZUMAETA,04-Oct,MIV,UNSECURED,CLASA MORA


In [28]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
no_multi_count

2

In [29]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], inplace=True)
df_multi.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
df_multi.reset_index(drop=True, inplace=True)
df_multi.to_excel(multiproducto, index=False)

multi_count_final = df_multi.shape[0]
print(multi_count_final)
df_multi.head(5)

5


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
0,2024-10-03,20275065,00248,100.0,PEN,MARIA EUGENIA SALAZAR SALVATIERRA PAGO,P,2,None,None,None,MARIA EUGENIA SALAZAR SALVATIERRA,04-Oct,MIV
1,2024-10-03,23589142,00248,4500.0,PEN,OLIMPO DAVID CARPIO ALVARADO,P,2,None,None,None,OLIMPO DAVID CARPIO ALVARADO,04-Oct,MIV
2,2024-10-03,25975753,00248,485.0,PEN,EDWIN ALEJANDRO PEREZ ZUMAETA 71531021*,P,2,None,None,None,EDWIN ALEJANDRO PEREZ ZUMAETA,04-Oct,MIV
3,2024-10-03,25975753,00248,6067.0,PEN,EDWIN ALEJANDRO PEREZ ZUMAETA 71531021,P,2,None,None,None,EDWIN ALEJANDRO PEREZ ZUMAETA,04-Oct,MIV
4,2024-10-03,30812996,00248,5500.0,PEN,INVERSIONES Y SERVICIOSGENERALESDUNASTV,P,3,None,None,None,INVERSIONES Y SERVICIOS GENERALES DUNASTV SAC,04-Oct,MIV


In [30]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

MULTIPRODUCTO OK


In [31]:
#################### NO ENVIADOS ####################

In [32]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].astype('Int64').fillna(0)

df_no_enviados['CONTRATO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['CONTRATO'], axis=1)
df_no_enviados['TIPO_FONDO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_FONDO'], axis=1)

df_no_enviados['FLAG'] = df_no_enviados['FLAG'].apply(lambda x: 'NE' if x == 0 else x)

df_no_enviados.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
df_no_enviados.reset_index(drop=True, inplace=True)
df_no_enviados.to_excel(no_enviados, index=False)

print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

(11, 17)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO         object
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG               object
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-10-03,13361975,00248,00083,1100.0,PEN,"DANIA MARIETA SARDON SANCHEZ,,,",P,1,001102009600556058,NULL,BC,DANIA MARIETA SARDON SANCHEZ,04-Oct,MIV,SECURED,SIN AGENCIA
1,2024-10-03,20862718,00249,00138,150.0,USD,MARIA ELENA GONZALEZ TENORIO..,P,1,001103289600014197,NULL,BC,MARIA ELENA GONZALEZ TENORIO,04-Oct,MIV,SECURED,SIN AGENCIA
2,2024-10-03,20982291,00248,00233,2500.0,PEN,JORGE ARMANADO GARCIA BARRIOS,P,1,001108319601006834,NULL,BC,JORGE ARMANDO GARCIA BARRIOS,04-Oct,MIV,SECURED,SIN AGENCIA
3,2024-10-03,22804438,00248,00215,5000.0,PEN,PAGO DEPRESTAMO 40286153 CELSA CERRON,P,1,001102155002554969,NULL,BC,CELSA CERRON MENDIETA,04-Oct,MIV,SECURED,SIN AGENCIA
4,2024-10-03,25686336,00248,00750,1000.0,PEN,JOSEPHAM RUIZ VENTURA,P,1,001107129600011802,NULL,BC,JOSEPHAM RUIZ VENTURA,04-Oct,MIV,SECURED,SIN AGENCIA


In [33]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

No enviados: 11
Enviados: 114
Total: 125
REGISTROS OK


In [34]:
if no_eviados == df_no_enviados.shape[0]:
    print('NO ENVIADOS OK')
else:
    print('NO ENVIADOS ERROR')

NO ENVIADOS OK


In [35]:
#################### FORMATO EXCEL ####################

In [36]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 15):  # Columnas L-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi_final':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 16):  # Columnas L-P
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'instruccion':
        for col in range(1, 11): # Columnas A-J
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(11, 13): # Columnas K-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 17):  # Columnas M-P
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(17, 18):  # Columna Q
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

format_excel(mono_path, "mono")
format_excel(multi_path, "multi")
format_excel(react_path, "instruccion")
format_excel(no_enviados_path, "instruccion")

In [37]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Abrir archivos?", icon='warning')
if result == 'yes':
    os.startfile(mono_path)
    os.startfile(multi_path)
    os.startfile(react_path)
    os.startfile(no_enviados_path)

In [38]:
print("monoproducto:", df_mono.shape)
print("multiproducto:", df_multi.shape)
print("reactiva:", df_reactiva.shape)
print("no enviados:", df_no_enviados.shape)

monoproducto: (105, 16)
multiproducto: (5, 14)
reactiva: (4, 17)
no enviados: (11, 17)


In [39]:
#################### AGENCIAS ####################

In [40]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar multiproducto agencias?", icon='warning')
if result == 'yes':
    df_1 = pd.read_excel(file_path+'/multi_agencias/RJ.xlsx')
    df_2 = pd.read_excel(file_path+'/multi_agencias/CLASA.xlsx')
    df_3 = pd.read_excel(file_path+'/multi_agencias/MORNESE.xlsx')
    
    df_1 = df_1[cols_multi]
    df_2 = df_2[cols_multi]
    df_3 = df_3[cols_multi]
    
    df_1['AGENCIA'] = 'ASESCOM RJ'
    df_2['AGENCIA'] = 'CLASA MORA'
    df_3['AGENCIA'] = 'MORNESE MORA'
    
    print("ASESCOM RJ:", df_1.shape)
    print("CLASA MORA:", df_2.shape)
    print("MORNESE MORA:", df_3.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    contratos_rj = df_1['CONTRATO'].notna().value_counts()[True] - 1
    contratos_clasa = df_2['CONTRATO'].notna().value_counts()[True] - 1
    contratos_mornese = df_3['CONTRATO'].notna().value_counts()[True] - 1
    
    print("Contratos RJ:", contratos_rj)
    print("Contratos CLASA:", contratos_clasa)
    print("Contratos MORNESE:", contratos_mornese)
    print("Total:", contratos_rj + contratos_clasa + contratos_mornese)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_agencias = pd.concat([df_1, df_2, df_3])
    df_agencias.dropna(subset=['CONTRATO'], inplace=True)
    df_agencias['CONTRATO'] = df_agencias['CONTRATO'].apply(
        lambda x: str(int(x)).replace(' ', '').zfill(18) 
        if 16 <= len(str(x).replace(' ', '')) <= 18 
        else None)
    df_agencias.dropna(subset=['CONTRATO'], inplace=True)
    df_agencias.reset_index(drop=True, inplace=True)
    
    df_multi = pd.read_excel(multi_path)
    df_multi['AGENCIA'] = 'NULL'
    
    print("Total Multiproducto Original:", df_multi.shape)
    print("Total Multiproducto Agencias:", df_agencias.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_multi_final = pd.concat([df_multi, df_agencias])
    
    df_multi_final['CC'] = df_multi_final['CC'].astype(str).replace(' ', '').astype('Int64').astype(str).str.zfill(8)
    df_multi_final.drop(columns=['TIPO_FONDO', 'CARTERA'], inplace=True)
    df_multi_final.sort_values(by=['CC', 'CONTRATO'], inplace=True)
    df_multi_final.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)
    df_multi_final.reset_index(drop=True, inplace=True)
    
    df_asignacion_test['CODIGO'] = df_asignacion_test['CODIGO'].astype('Int64').astype(str).str.zfill(8)
    df_asignacion_test['CONTRATO'] = df_asignacion_test['CONTRATO'].apply(
        lambda x: str(int(x)).replace(' ', '').zfill(18) if pd.notna(x) else x)
    
    df_multi_final = df_multi_final.merge(df_asignacion_test, on="CONTRATO", how="left")
    df_multi_final['CLAVSERV'] = df_multi_final['CLAVSERV'].astype(str).str.zfill(5)
    df_multi_final['CONTRATO'] = df_multi_final['CONTRATO'].fillna('NULL')
    df_multi_final['TIPO_FONDO'] = df_multi_final['TIPO_FONDO'].fillna('NULL')
    df_multi_final['CARTERA'] = df_multi_final['CARTERA'].fillna('NULL')
    df_multi_final['TIPO_CARTERA'] = df_multi_final['TIPO_CARTERA'].fillna('NULL')
    df_multi_final['AGENCIA_y'] = df_multi_final['AGENCIA_y'].fillna('NULL')
    
    df_multi_final['AGENCIA_y'] = df_multi_final.apply(
        lambda x: x['AGENCIA_y'] 
        if x['AGENCIA_y'] == x['AGENCIA_x'] else f"{x['AGENCIA_x']}/{x['AGENCIA_y']}", 
        axis=1)
    
    df_multi_final.drop(columns=['AGENCIA_x', 'FLAG_y', 'NOMBRE_CLIENTE_y'], inplace=True)
    df_multi_final.rename(columns={'AGENCIA_y': 'AGENCIA', 'FLAG_x': 'FLAG', 'NOMBRE_CLIENTE_x': 'NOMBRE_CLIENTE'}, inplace=True)
    
    cols_multi_final = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
    df_multi_final = df_multi_final[cols_multi_final]
    df_multi_final.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_multi_final.reset_index(drop=True, inplace=True)
    
    print("Multiproducto final:", df_multi_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_no_enviados_multi = df_multi_final[(df_multi_final['CONTRATO'] == 'NULL') | (df_multi_final['TIPO_CARTERA'] != 'UNSECURED')]
    df_no_enviados_multi['AGENCIA'] = df_no_enviados_multi.apply(
        lambda x: 'AGENCIA NO IDENTIFICA' if x['CONTRATO'] == 'NULL' else x['AGENCIA'], axis=1)
    print("No enviados Multiproducto:", df_no_enviados_multi.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_reactivas_multi = df_multi_final[(df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & (df_multi_final['TIPO_FONDO'] != 'NULL')]
    print("Reactivas Multiproducto:", df_reactivas_multi.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_multi_final = df_multi_final[(df_multi_final['CONTRATO'] != 'NULL') & 
                                    (df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & 
                                    (df_multi_final['TIPO_FONDO'] == 'NULL')]
    df_multi_final.to_excel(multiproducto, index=False)
    
    print("Total Multiproducto:", df_multi_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_no_enviados_final = pd.concat([df_no_enviados, df_no_enviados_multi])
    
    df_no_enviados_final['CONTRATO'] = df_no_enviados_final['CONTRATO'].fillna('NULL')
    df_no_enviados_final['TIPO_FONDO'] = df_no_enviados_final['TIPO_FONDO'].fillna('NULL')
    df_no_enviados_final['CARTERA'] = df_no_enviados_final['CARTERA'].fillna('NULL')
    df_no_enviados_final['TIPO_CARTERA'] = df_no_enviados_final['TIPO_CARTERA'].fillna('NULL')
    df_no_enviados_final['NOMBRE_CLIENTE'] = df_no_enviados_final['NOMBRE_CLIENTE'].fillna('NULL')
    df_no_enviados_final['AGENCIA'] = df_no_enviados_final['AGENCIA'].fillna('NULL')
    df_no_enviados_final['CLAVSERV'] = df_no_enviados_final['CLAVSERV'].apply(lambda x: str(int(x)).replace(' ', '').zfill(5))
    df_no_enviados_final.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_no_enviados_final.reset_index(drop=True, inplace=True)
    df_no_enviados_final.to_excel(no_enviados, index=False)
    
    print("Total NO Enviados:", df_no_enviados_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_reactiva_final = pd.concat([df_reactiva, df_reactivas_multi])
    df_reactiva_final.to_excel(reactiva, index=False)
    
    print("Total Reactiva:", df_reactiva_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_enviados = pd.concat([df_mono, df_multi_final])
    
    df_base_test_final = df_base_test[['CC', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE']]
    df_base_test_final['KEY'] = df_base_test_final['CC'].astype(str) + df_base_test_final['IMPORTE'].astype(str) + df_base_test_final['MONEDA'] + df_base_test_final['NOMBRE']
    df_enviados['KEY'] = df_enviados['CC'].astype(str) + df_enviados['IMPORTE'].astype(str) + df_enviados['MONEDA'] + df_enviados['NOMBRE']
    
    df_enviados_final = df_enviados.merge(df_base_test_final[['KEY', 'CENTROPAGO']], on='KEY', how='left')
    df_enviados_final['CLAVSERV'] = df_enviados_final['CLAVSERV'].apply(lambda x: str(int(x)).zfill(5))
    df_enviados_final.drop(columns=['KEY'], inplace=True)
    df_enviados_final.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_enviados_final.reset_index(drop=True, inplace=True)
    
    cols_enviados = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
    df_enviados_final = df_enviados_final[cols_enviados]
    df_enviados_final.to_excel(si_enviados, index=False)
    
    print("Total Enviados:", df_enviados_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    format_excel(multi_path, "multi_final")
    format_excel(react_path, "instruccion")
    format_excel(no_enviados_path, "instruccion")
    format_excel(enviados_path, "instruccion")
    
    os.startfile(multi_path)
    os.startfile(react_path)
    os.startfile(no_enviados_path)
    os.startfile(enviados_path)